In [1]:
# Import
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from finta import TA
from talib import RSI, BBANDS

# Allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Imports required for automated plot to PNG saves (one time use)

# import holoviews as hv
# hv.extension('bokeh')

# # selenium 4
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [3]:
# Read in the CSV files that contain SMA, EMA, and BB/RSI inidcators based on ratio data
googl_df = pd.read_csv("Resources/googl_ratio_indicators.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
nvda_df = pd.read_csv("Resources/nvda_ratio_indicators.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
mmm_df = pd.read_csv("Resources/mmm_ratio_indicators.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
pg_df = pd.read_csv("Resources/pg_ratio_indicators.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)

In [4]:
# Create a list of tuples containing the stock DataFrames and their respective columns for iteration
ticker_data_all = [
    (googl_df, [("GOOGL P/S (LTM)", "ps"), ("GOOGL P/FCF (LTM)", "pfcf"), ("GOOGL P/E (LTM)", "pe")]),
    (nvda_df, [("NVDA P/S (LTM)", "ps"), ("NVDA P/FCF (LTM)", "pfcf"), ("NVDA P/E (LTM)", "pe")]),
    (mmm_df, [("MMM P/S (LTM)", "ps"), ("MMM P/FCF (LTM)", "pfcf"), ("MMM P/E (LTM)", "pe")]),
    (pg_df, [("PG P/S (LTM)", "ps"), ("PG P/FCF (LTM)", "pfcf"), ("PG P/E (LTM)", "pe")]),
]

In [5]:
# Create and Populate the "signal" column for signal DataFrames
for df, metrics in ticker_data_all:
    for column_name, prefix, in metrics:
        df[f'{prefix}_Signal_sma'] = 0.0
        df[f'{prefix}_Signal_ema'] = 0.0
        df[f'{prefix}_Signal_sma30'] = 0.0
        df[f'{prefix}_Signal_ema30'] = 0.0

In [6]:
# Generate the trading signal 0 or 1,
# where 1 is when sma_fast is greater than sma_slow or ema_fast is greater than ema_slow
# and 0 otherwise
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        # Apply the condition to set the 'Signal' column
        df[f'{prefix}_Signal_sma'] = np.where(df[f'{prefix}_sma_slow'] > df[f'{prefix}_sma_fast'], 1.0, 0.0)
        df[f'{prefix}_Signal_ema'] = np.where(df[f'{prefix}_ema_slow'] > df[f'{prefix}_ema_fast'], 1.0, 0.0)
        df[f'{prefix}_Signal_sma30'] = np.where(df[f'{prefix}_sma_slow'] > df[f'{prefix}_sma_fast30'], 1.0, 0.0)
        df[f'{prefix}_Signal_ema30'] = np.where(df[f'{prefix}_ema_slow'] > df[f'{prefix}_ema_fast30'], 1.0, 0.0)

# Note: SMA and EMA fast and slow signals positions have been reversed in the signal equasion to reflect the fact that
# Note: they have been calculated based on the ratio data (P/S, P/FCF, P/E) where a lower value is preferred

In [7]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Entry/Exit_sma'] = df[f'{prefix}_Signal_sma'].diff()
        df[f'{prefix}_Entry/Exit_ema'] = df[f'{prefix}_Signal_ema'].diff()
        df[f'{prefix}_Entry/Exit_sma30'] = df[f'{prefix}_Signal_sma30'].diff()
        df[f'{prefix}_Entry/Exit_ema30'] = df[f'{prefix}_Signal_ema30'].diff()

### SMA (10/100, from ratios) Entry/Exit Plots

In [8]:
# Initialize a dictionary to store the plots for SMA strategy from each ratio for each stock
plots_sma = {}

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        
        # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
        stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        
        # Plot exit positions
        exit = df[df[f'{prefix}_Entry/Exit_sma'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
            color='orange',
            marker='v',
            size=200,
            legend=False,
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot entry positions
        entry = df[df[f'{prefix}_Entry/Exit_sma'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
            color='purple',
            marker='^',
            size=200,
            legend=False,
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot security close price
        security_close = df[[f'{stock_symbol} Adj. Close']].hvplot(
            line_color='lightgray',
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot moving averages
        moving_avgs = df[[f'{prefix}_sma_fast', f'{prefix}_sma_slow']].hvplot(
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Create the overlay plot
        entry_exit_plot = security_close * moving_avgs * entry * exit

        # Set the title dynamically based on the stock symbol
        entry_exit_plot = entry_exit_plot.opts(
            title=f"{stock_symbol} - SMA10, SMA100, Entry and Exit Points - from {prefix.upper()} Ratio"
        )

        # Store the plot in the dictionary using the stock symbol and ratio prefix as key
        
        plots_sma[f"{stock_symbol}_{prefix}"] = entry_exit_plot

### SMA (30/100, from ratios) Entry/Exit Plots

In [9]:
# Initialize a dictionary to store the plots for SMA strategy from each ratio for each stock
plots_sma30 = {}

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        
        # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
        stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        
        # Plot exit positions
        exit = df[df[f'{prefix}_Entry/Exit_sma30'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
            color='orange',
            marker='v',
            size=200,
            legend=False,
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot entry positions
        entry = df[df[f'{prefix}_Entry/Exit_sma30'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
            color='purple',
            marker='^',
            size=200,
            legend=False,
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot security close price
        security_close = df[[f'{stock_symbol} Adj. Close']].hvplot(
            line_color='lightgray',
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot moving averages
        moving_avgs = df[[f'{prefix}_sma_fast30', f'{prefix}_sma_slow']].hvplot(
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Create the overlay plot
        entry_exit_plot = security_close * moving_avgs * entry * exit

        # Set the title dynamically based on the stock symbol
        entry_exit_plot = entry_exit_plot.opts(
            title=f"{stock_symbol} - SMA30, SMA100, Entry and Exit Points - from {prefix.upper()} Ratio"
        )

        # Store the plot in the dictionary using the stock symbol and ratio prefix as key
        
        plots_sma30[f"{stock_symbol}_{prefix}"] = entry_exit_plot

### EMA (10/100, from ratios) Entry/Exit Plots

In [10]:
# Initialize a dictionary to store the plots for SMA strategy from each ratio for each stock
plots_ema = {}

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        
        # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
        stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        
        # Plot exit positions
        exit = df[df[f'{prefix}_Entry/Exit_ema'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
            color='orange',
            marker='v',
            size=200,
            legend=False,
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot entry positions
        entry = df[df[f'{prefix}_Entry/Exit_ema'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
            color='purple',
            marker='^',
            size=200,
            legend=False,
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot security close price
        security_close = df[[f'{stock_symbol} Adj. Close']].hvplot(
            line_color='lightgray',
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot moving averages
        moving_avgs = df[[f'{prefix}_ema_fast', f'{prefix}_ema_slow']].hvplot(
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Create the overlay plot
        entry_exit_plot = security_close * moving_avgs * entry * exit

        # Set the title dynamically based on the stock symbol
        entry_exit_plot = entry_exit_plot.opts(
            title=f"{stock_symbol} - EMA10, EMA100, Entry and Exit Points - from {prefix.upper()} Ratio"
        )

        # Store the plot in the dictionary using the stock symbol and ratio prefix as key
        
        plots_ema[f"{stock_symbol}_{prefix}"] = entry_exit_plot

### EMA (30/100, from ratios) Entry/Exit Plots

In [11]:
# Initialize a dictionary to store the plots for SMA strategy from each ratio for each stock
plots_ema30 = {}

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        
        # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
        stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        
        # Plot exit positions
        exit = df[df[f'{prefix}_Entry/Exit_ema30'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
            color='orange',
            marker='v',
            size=200,
            legend=False,
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot entry positions
        entry = df[df[f'{prefix}_Entry/Exit_ema30'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
            color='purple',
            marker='^',
            size=200,
            legend=False,
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot security close price
        security_close = df[[f'{stock_symbol} Adj. Close']].hvplot(
            line_color='lightgray',
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Plot moving averages
        moving_avgs = df[[f'{prefix}_ema_fast30', f'{prefix}_ema_slow']].hvplot(
            ylabel='Price in $',
            width=1000,
            height=400
        )

        # Create the overlay plot
        entry_exit_plot = security_close * moving_avgs * entry * exit

        # Set the title dynamically based on the stock symbol
        entry_exit_plot = entry_exit_plot.opts(
            title=f"{stock_symbol} - EMA30, EMA100, Entry and Exit Points - from {prefix.upper()} Ratio"
        )

        # Store the plot in the dictionary using the stock symbol and ratio prefix as key
        
        plots_ema30[f"{stock_symbol}_{prefix}"] = entry_exit_plot

### Investment Capital Tracking

In [12]:
# Set initial investment capital
initial_capital = float(100000)

# Set the share size per transaction
share_size = 500

In [13]:
# Creating a position column to store the number of shares held
# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Position_sma'] = share_size * df[f'{prefix}_Signal_sma']
        df[f'{prefix}_Position_ema'] = share_size * df[f'{prefix}_Signal_ema']
        df[f'{prefix}_Position_sma30'] = share_size * df[f'{prefix}_Signal_sma30']
        df[f'{prefix}_Position_ema30'] = share_size * df[f'{prefix}_Signal_ema30']

In [14]:
# Determine the points in time where the share position is bought or sold
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Entry/Exit Position_sma'] = df[f'{prefix}_Position_sma'].diff()
        df[f'{prefix}_Entry/Exit Position_ema'] = df[f'{prefix}_Position_ema'].diff()
        df[f'{prefix}_Entry/Exit Position_sma30'] = df[f'{prefix}_Position_sma30'].diff()
        df[f'{prefix}_Entry/Exit Position_ema30'] = df[f'{prefix}_Position_ema30'].diff()

In [15]:
# Multiply the close price by the number of shares held, or the Position
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        adj_close_col = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        df[f'{prefix}_Portfolio Holdings_sma'] = df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Position_sma']
        df[f'{prefix}_Portfolio Holdings_ema'] = df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Position_ema']
        df[f'{prefix}_Portfolio Holdings_sma30'] = df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Position_sma30']
        df[f'{prefix}_Portfolio Holdings_ema30'] = df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Position_ema30']

In [16]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        adj_close_col = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        df[f'{prefix}_Portfolio Cash_sma'] = initial_capital - (df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Entry/Exit Position_sma']).cumsum()
        df[f'{prefix}_Portfolio Cash_ema'] = initial_capital - (df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Entry/Exit Position_ema']).cumsum()
        df[f'{prefix}_Portfolio Cash_sma30'] = initial_capital - (df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Entry/Exit Position_sma30']).cumsum()
        df[f'{prefix}_Portfolio Cash_ema30'] = initial_capital - (df[f'{adj_close_col} Adj. Close'] * df[f'{prefix}_Entry/Exit Position_ema30']).cumsum()

In [17]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Portfolio Total_sma'] = df[f'{prefix}_Portfolio Cash_sma'] + df[f'{prefix}_Portfolio Holdings_sma']
        df[f'{prefix}_Portfolio Total_ema'] = df[f'{prefix}_Portfolio Cash_ema'] + df[f'{prefix}_Portfolio Holdings_ema']
        df[f'{prefix}_Portfolio Total_sma30'] = df[f'{prefix}_Portfolio Cash_sma30'] + df[f'{prefix}_Portfolio Holdings_sma30']
        df[f'{prefix}_Portfolio Total_ema30'] = df[f'{prefix}_Portfolio Cash_ema30'] + df[f'{prefix}_Portfolio Holdings_ema30']

In [18]:
# Calculate the portfolio daily returns
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Portfolio Daily Returns_sma'] = df[f'{prefix}_Portfolio Total_sma'].pct_change()
        df[f'{prefix}_Portfolio Daily Returns_ema'] = df[f'{prefix}_Portfolio Total_ema'].pct_change()
        df[f'{prefix}_Portfolio Daily Returns_sma30'] = df[f'{prefix}_Portfolio Total_sma30'].pct_change()
        df[f'{prefix}_Portfolio Daily Returns_ema30'] = df[f'{prefix}_Portfolio Total_ema30'].pct_change()

In [19]:
# Calculate the portfolio cumulative returns
for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        df[f'{prefix}_Portfolio Cumulative Returns_sma'] = (1 + df[f'{prefix}_Portfolio Daily Returns_sma']).cumprod() - 1
        df[f'{prefix}_Portfolio Cumulative Returns_ema'] = (1 + df[f'{prefix}_Portfolio Daily Returns_ema']).cumprod() - 1
        df[f'{prefix}_Portfolio Cumulative Returns_sma30'] = (1 + df[f'{prefix}_Portfolio Daily Returns_sma30']).cumprod() - 1
        df[f'{prefix}_Portfolio Cumulative Returns_ema30'] = (1 + df[f'{prefix}_Portfolio Daily Returns_ema30']).cumprod() - 1

### SMA (10/100, from ratios) Portfolio Value Plots for each ticker

In [20]:
# Initialize a dictionary to store the plots
plots_sma_portfolio_value = {}

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        
        # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
        stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        
        # Visualize exit position relative to total portfolio value
        exit = df[df[f'{prefix}_Entry/Exit_sma'] == -1.0][f'{prefix}_Portfolio Total_sma'].hvplot.scatter(
            color='orange',
            marker='v',
            size=200,
            legend=False,
            ylabel='Total Portfolio Value',
            width=1000,
            height=400
        )

        # Visualize entry position relative to total portfolio value
        entry = df[df[f'{prefix}_Entry/Exit_sma'] == 1.0][f'{prefix}_Portfolio Total_sma'].hvplot.scatter(
            color='purple',
            marker='^',
            size=200,
            ylabel='Total Portfolio Value',
            width=1000,
            height=400
        )

        # Visualize the value of the total portfolio
        total_portfolio_value = df[[f'{prefix}_Portfolio Total_sma']].hvplot(
            line_color='lightgray',
            ylabel='Total Portfolio Value',
            xlabel='Date',
            width=1000,
            height=400
        )

        # Overlay the plots
        portfolio_entry_exit_plot = total_portfolio_value * entry * exit

        # Set the title dynamically based on the stock symbol
        portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
            title=f"{stock_symbol} - SMA10, SMA100, Total Portfolio Value - from {prefix.upper()} Ratio",
            yformatter='%.0f'
        )

        # Store the plot in the dictionary using the stock symbol as key
        plots_sma_portfolio_value[f"{stock_symbol}_{prefix}"] = portfolio_entry_exit_plot

### SMA (30/100, from ratios) Portfolio Value Plots for each ticker

In [21]:
# Initialize a dictionary to store the plots
plots_sma30_portfolio_value = {}

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        
        # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
        stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        
        # Visualize exit position relative to total portfolio value
        exit = df[df[f'{prefix}_Entry/Exit_sma30'] == -1.0][f'{prefix}_Portfolio Total_sma30'].hvplot.scatter(
            color='orange',
            marker='v',
            size=200,
            legend=False,
            ylabel='Total Portfolio Value',
            width=1000,
            height=400
        )

        # Visualize entry position relative to total portfolio value
        entry = df[df[f'{prefix}_Entry/Exit_sma30'] == 1.0][f'{prefix}_Portfolio Total_sma30'].hvplot.scatter(
            color='purple',
            marker='^',
            size=200,
            ylabel='Total Portfolio Value',
            width=1000,
            height=400
        )

        # Visualize the value of the total portfolio
        total_portfolio_value = df[[f'{prefix}_Portfolio Total_sma30']].hvplot(
            line_color='lightgray',
            ylabel='Total Portfolio Value',
            xlabel='Date',
            width=1000,
            height=400
        )

        # Overlay the plots
        portfolio_entry_exit_plot = total_portfolio_value * entry * exit

        # Set the title dynamically based on the stock symbol
        portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
            title=f"{stock_symbol} - SMA30, SMA100, Total Portfolio Value - from {prefix.upper()} Ratio",
            yformatter='%.0f'
        )

        # Store the plot in the dictionary using the stock symbol as key
        plots_sma30_portfolio_value[f"{stock_symbol}_{prefix}"] = portfolio_entry_exit_plot

### EMA (10/100, from ratios) Portfolio Value Plots for each ticker

In [22]:
# Initialize a dictionary to store the plots
plots_ema_portfolio_value = {}

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        
        # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
        stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        
        # Visualize exit position relative to total portfolio value
        exit = df[df[f'{prefix}_Entry/Exit_ema'] == -1.0][f'{prefix}_Portfolio Total_ema'].hvplot.scatter(
            color='orange',
            marker='v',
            size=200,
            legend=False,
            ylabel='Total Portfolio Value',
            width=1000,
            height=400
        )

        # Visualize entry position relative to total portfolio value
        entry = df[df[f'{prefix}_Entry/Exit_ema'] == 1.0][f'{prefix}_Portfolio Total_ema'].hvplot.scatter(
            color='purple',
            marker='^',
            size=200,
            ylabel='Total Portfolio Value',
            width=1000,
            height=400
        )

        # Visualize the value of the total portfolio
        total_portfolio_value = df[[f'{prefix}_Portfolio Total_ema']].hvplot(
            line_color='lightgray',
            ylabel='Total Portfolio Value',
            xlabel='Date',
            width=1000,
            height=400
        )

        # Overlay the plots
        portfolio_entry_exit_plot = total_portfolio_value * entry * exit

        # Set the title dynamically based on the stock symbol
        portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
            title=f"{stock_symbol} - EMA10, EMA100, Total Portfolio Value - from {prefix.upper()} Ratio",
            yformatter='%.0f'
        )

        # Store the plot in the dictionary using the stock symbol as key
        plots_ema_portfolio_value[f"{stock_symbol}_{prefix}"] = portfolio_entry_exit_plot

### EMA (30/100, from ratios) Portfolio Value Plots for each ticker

In [23]:
# Initialize a dictionary to store the plots
plots_ema30_portfolio_value = {}

for df, metrics in ticker_data_all:
    for column_name, prefix in metrics:
        
        # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
        stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]
        
        # Visualize exit position relative to total portfolio value
        exit = df[df[f'{prefix}_Entry/Exit_ema30'] == -1.0][f'{prefix}_Portfolio Total_ema30'].hvplot.scatter(
            color='orange',
            marker='v',
            size=200,
            legend=False,
            ylabel='Total Portfolio Value',
            width=1000,
            height=400
        )

        # Visualize entry position relative to total portfolio value
        entry = df[df[f'{prefix}_Entry/Exit_ema30'] == 1.0][f'{prefix}_Portfolio Total_ema30'].hvplot.scatter(
            color='purple',
            marker='^',
            size=200,
            ylabel='Total Portfolio Value',
            width=1000,
            height=400
        )

        # Visualize the value of the total portfolio
        total_portfolio_value = df[[f'{prefix}_Portfolio Total_ema30']].hvplot(
            line_color='lightgray',
            ylabel='Total Portfolio Value',
            xlabel='Date',
            width=1000,
            height=400
        )

        # Overlay the plots
        portfolio_entry_exit_plot = total_portfolio_value * entry * exit

        # Set the title dynamically based on the stock symbol
        portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
            title=f"{stock_symbol} - EMA30, EMA100, Total Portfolio Value - from {prefix.upper()} Ratio",
            yformatter='%.0f'
        )

        # Store the plot in the dictionary using the stock symbol as key
        plots_ema30_portfolio_value[f"{stock_symbol}_{prefix}"] = portfolio_entry_exit_plot

### Save all stock DataFrames as CSVs for training models

In [24]:
googl_df.shape

(1320, 132)

In [ ]:
# Write individual stock DFs with signals and capital tracking to CSV files
googl_df.to_csv('Resources/googl_signals.csv', index=True)
nvda_df.to_csv('Resources/nvda_signals.csv', index=True)
mmm_df.to_csv('Resources/mmm_signals.csv', index=True)
pg_df.to_csv('Resources/pg_signals.csv', index=True)

### Display All Plots

In [ ]:
googl_df.columns

In [ ]:
# Plot the SMA (10/100,from each ratio) entry and exit points for each stock
# Note: This will display 12 plots (3 per stock)
for plot in plots_sma.values():
    display(plot)

# Generate png files for the plots automatically (one time use)
# for i, plot in enumerate(plots_sma.values(), start=1):
#     hv.save(plot, f'plot_{i}.png', fmt='png')

In [ ]:
# Plot the SMA (30/100,from each ratio) entry and exit points for each stock
# Note: This will display 12 plots (3 per stock)
for plot in plots_sma30.values():
    display(plot)

# Generate png files for the plots automatically (one time use)
# for i, plot in enumerate(plots_sma30.values(), start=13):
#     hv.save(plot, f'plot_{i}.png', fmt='png')

In [ ]:
# Plot the EMA (10/100,from each ratio) entry and exit points for each stock
# Note: This will display 12 plots (3 per stock)
for plot in plots_ema.values():
    display(plot)

In [ ]:
# Plot the EMA (30/100,from each ratio) entry and exit points for each stock
# Note: This will display 12 plots (3 per stock)
for plot in plots_ema30.values():
    display(plot)

In [ ]:
# Plot the SMA (10/100,from each ratio) Total Portfolio Value for each stock
# Note: This will display 12 plots (3 per stock)
for plot in plots_sma_portfolio_value.values():
    display(plot)

In [ ]:
# Plot the SMA (30/100,from each ratio) Total Portfolio Value for each stock
# Note: This will display 12 plots (3 per stock)
for plot in plots_sma30_portfolio_value.values():
    display(plot)

In [ ]:
# Plot the EMA (10/100,from each ratio) Total Portfolio Value for each stock
# Note: This will display 12 plots (3 per stock)
for plot in plots_ema_portfolio_value.values():
    display(plot)

In [ ]:
# Plot the EMA (30/100,from each ratio) Total Portfolio Value for each stock
# Note: This will display 12 plots (3 per stock)
for plot in plots_ema30_portfolio_value.values():
    display(plot)